## ProjF5 - Final Model

* **Please be advised that all models have been re-executed, and as a result, there may be slight variations in the outcomes compared to those presented in the slides.**

* **Our models are LSTM+CNN and BiLSTM+CNN**






In [1]:
#Importing all the necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from keras.optimizers import Adam
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Bidirectional, LSTM, Conv1D, MaxPooling1D, Flatten, Concatenate, Dense, GRU
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

2024-10-21 12:53:29.334397: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-21 12:53:29.337555: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-21 12:53:29.346313: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-21 12:53:29.361051: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-21 12:53:29.365278: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-21 12:53:29.377832: I tensorflow/core/platform/cpu_feature_gu

### 1. Load and Prepare Data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [75]:
#Reading data as a DataFrame
data = pd.read_csv('/content/drive/MyDrive/NNDL/sales_prediction_data.csv')

In [ ]:
data

In [ ]:
#Checking for duplicate rows in the dataset
print("Duplicate rows:", data.duplicated().sum())

In [78]:
#Converting 'date' attribute to date type
data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')


In [ ]:
#Checking for null values in the dataset
data.isna().sum()

In [ ]:
#Correlation of each attribute with sales
corr = data.corr()
corr['sales'].sort_values(ascending=False)


Since none of the attributes individually exhibit a strong correlation with sales, we have decided to include all attributes in the model. By considering them together, we aim to capture potential interactions and dependencies between the attributes, which can contribute to the effectiveness of the model during training.

In [81]:
'''Sorting the data according to date as it is a time-stamped dataset and it
would be convinient for traininig the models on sorted data'''

data = data.sort_values('date')

In [ ]:
'''It can be seen that all the null values for sales are from 2018-01-01 to
2018-03-31 as the goal of the competition, from where this dataset is obtained,
is to predict the sales of the stores during this time period using historical
data'''

data[data['sales'].isna()]

In [83]:
#Storing the rows where sales need to be predicted in y (test set)
y = data[data['sales'].isna()]


In [ ]:
y

In [85]:
data = data.drop(y.index)

In [ ]:
data.isna().sum()

In [87]:
#Storing the rest of the data in X (train set)
X = data
X = X.reset_index(drop=True)

In [ ]:
#Printing the train set
X

In [ ]:
#Printing the test set
y = y.reset_index(drop=True)
y

In [ ]:
#Data distribution for each year in train set
X['year'] = X['date'].dt.year


year_counts = X['year'].value_counts()

plt.pie(year_counts, labels=year_counts.index, autopct='%1.1f%%')


plt.title('Distribution of Data Values by Year')


plt.show()


As we can see, we do not have any data imbalance in the train set. The data is distributed uniformly for each year.

In [ ]:
#Bar plot for showing the data distribution in train and test split

num_train = X.shape[0]
num_test = y.shape[0]

plt.bar(['Training Set', 'Test Set'], [num_train, num_test])

plt.title('Number of Data Values in Each Set')
plt.xlabel('Set')
plt.ylabel('Number of Data Values')

plt.show()


### 2. Preparing your Final Model

#BiLSTM

In [92]:
X_bilstm = X

In [ ]:
X_bilstm

In [94]:
# Converting 'date' column to ordinal to use it as a feature
X_bilstm['date'] = X_bilstm['date'].apply(lambda x: x.toordinal())

In [ ]:
X_bilstm

In [99]:
# Dropping 'year' column as it is redundant
X_bilstm.drop(columns=['year'], inplace=True)

In [97]:
# Feature scaling
scaler = MinMaxScaler()
X_bilstm[['date', 'store', 'item']] = scaler.fit_transform(X_bilstm[['date', 'store', 'item']])
y = scaler.fit_transform(X_bilstm[['sales']])

In [ ]:
# Defining function to create sequences
def create_sequences(X, y, time_steps=5):
    X_seq, y_seq = [], []
    for i in range(len(X) - time_steps):
        X_seq.append(X[i:(i + time_steps)])
        y_seq.append(y[i + time_steps])
    return np.array(X_seq), np.array(y_seq)

In [ ]:
# Creating sequences for BiLSTM model
X_seq, y_seq = create_sequences(X_bilstm[['date', 'store', 'item']].values, y)

# Splitting the dataset into train and test sets
X_train_bilstm, X_test_bilstm, y_train_bilstm, y_test_bilstm = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

In [ ]:
#Using the same number of BiLSTM units as LSTM used in Baseline report for fair comparision

bilstm_model = Sequential([
    Bidirectional(LSTM(units=50, activation='relu'), input_shape=(X_train_bilstm.shape[1], X_train_bilstm.shape[2])),
    Dense(units=1)
])
bilstm_model.compile(optimizer='adam', loss='mse')

In [ ]:
# Training the model (using the same epoch and batch size as Baseline for fair comparision)
bilstm_model.fit(X_train_bilstm, y_train_bilstm, epochs=25, batch_size=32, validation_split=0.1)

In [ ]:
from tensorflow.keras.models import load_model
# Save the model
bilstm_model.save("bilstm_model.h5")


In [ ]:
bisltm_model = load_model("/content/bilstm_model.h5")

In [ ]:
# Evaluating the model
test_loss_bilstm = bisltm_model.evaluate(X_test_bilstm, y_test_bilstm)
print("Test Loss:", test_loss_bilstm)

#LSTM+CNN (Our Model)

In [21]:
X_hyb = X

In [22]:
# Converting 'date' column to ordinal to use it as a feature
X_hyb['date'] = X_hyb['date'].apply(lambda x: x.toordinal())

In [23]:
# Dropping 'year' column as it is redundant
X_hyb.drop(columns=['year'], inplace=True)

In [24]:
# # One-hot encode 'store' column
# one_hot_encoded = pd.get_dummies(X_hyb['store'], prefix='store')

# # Convert True/False to 1/0
# one_hot_encoded = one_hot_encoded.astype(int)

# # Concatenate one-hot encoded columns to the original DataFrame
# X_hyb = pd.concat([X_hyb.drop('store', axis=1), one_hot_encoded], axis=1)

In [ ]:
X_hyb

In [26]:
# Feature scaling
scaler = MinMaxScaler()
X_hyb.loc[:, X_hyb.columns != 'sales'] = scaler.fit_transform(X_hyb.loc[:, X_hyb.columns != 'sales'])
y = scaler.fit_transform(X_hyb[['sales']])

In [27]:
# Defining function to create sequences
def create_sequences(X, y, time_steps=5):
    X_seq, y_seq = [], []
    for i in range(len(X) - time_steps):
        X_seq.append(X[i:(i + time_steps)])
        y_seq.append(y[i + time_steps])
    return np.array(X_seq), np.array(y_seq)

In [28]:
# Creating sequences for LSTM+CNN model
X_seq, y_seq = create_sequences(X_hyb[['date', 'store', 'item']].values, y)

# Splitting the dataset into train and test sets
X_train_hyb, X_test_hyb, y_train_hyb, y_test_hyb = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

In [ ]:
y_train_hyb

In [ ]:
# Define input layers for LSTM and CNN models
lstm_input = Input(shape=(X_train_hyb.shape[1], X_train_hyb.shape[2]))
cnn_input = Input(shape=(X_train_hyb.shape[1], X_train_hyb.shape[2]))

# LSTM model
lstm_output = LSTM(units=50, activation='relu')(lstm_input)

# CNN model
cnn_output = Conv1D(filters=64, kernel_size=3, activation='relu')(cnn_input)
cnn_output = MaxPooling1D(pool_size=2)(cnn_output)
cnn_output = Flatten()(cnn_output)

# Concatenate the outputs of LSTM and CNN models
concatenated = Concatenate()([lstm_output, cnn_output])

# Combine with dense layers
combined = Dense(64, activation='relu')(concatenated)
output = Dense(1)(combined)

# Define the ensemble model
ensemble_model = Model(inputs=[lstm_input, cnn_input], outputs=output)

# Compile the model
ensemble_model.compile(optimizer='adam', loss='mse')

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint("ensemble_model_checkpoint.h5", monitor='val_loss', save_best_only=True)

# Train the model
history = ensemble_model.fit([X_train_hyb, X_train_hyb], y_train_hyb, epochs=50, batch_size=256, validation_data=([X_test_hyb, X_test_hyb], y_test_hyb), callbacks=[early_stopping, checkpoint])



In [31]:
# # Load the best model from checkpoint
# from tensorflow.keras.models import load_model
# hyb_model = load_model("ensemble_model_checkpoint.h5")
# hyb_model.save("ensemble_model.h5")


In [ ]:
# Evaluate the model on the test set
test_loss_ensemble = ensemble_model.evaluate([X_test_hyb, X_test_hyb], y_test_hyb)

print("Test Loss:", test_loss_ensemble)


#BiLSTM+CNN (Our Model)

In [52]:
X_bcnn = X

In [ ]:
X_bcnn

In [54]:
# Converting 'date' column to ordinal to use it as a feature
X_bcnn['date'] = X_bcnn['date'].apply(lambda x: x.toordinal())

In [55]:
# Dropping 'year' column as it is redundant
X_bcnn.drop(columns=['year'], inplace=True)

In [56]:
# X_encoded = pd.get_dummies(X_gru, columns=['store', 'item'], dtype=int)

# # Drop the original 'date' column if you don't need it
# X_encoded.drop(columns=['date'], inplace=True)

In [ ]:
X_bcnn

In [58]:
# Feature scaling
scaler = MinMaxScaler()

X_scaled = X_bcnn.iloc[:, :-1]

X_scaled.iloc[:, :] = scaler.fit_transform(X_scaled)

X_bcnn.iloc[:, :-1] = X_scaled
y = scaler.fit_transform(X_bcnn[['sales']])

In [ ]:
X_bcnn

In [60]:
# Defining function to create sequences
def create_sequences(X, y, time_steps=5):
    X_seq, y_seq = [], []
    for i in range(len(X) - time_steps):
        X_seq.append(X[i:(i + time_steps)])
        y_seq.append(y[i + time_steps])
    return np.array(X_seq), np.array(y_seq)

In [61]:
# Creating sequences for GRU+CNN model
X_seq, y_seq = create_sequences(X_bcnn.drop(columns=['sales']).values, y)

# Splitting the dataset into train and test sets
X_train_bcnn, X_test_bcnn, y_train_bcnn, y_test_bcnn = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

In [ ]:
# Define input layers for BiLSTM and CNN models
bilstm_input = Input(shape=(X_train_bcnn.shape[1], X_train_bcnn.shape[2]))
cnn_input = Input(shape=(X_train_bcnn.shape[1], X_train_bcnn.shape[2]))

# BiLSTM model
lstm_layer = LSTM(units=50, activation='relu')
bilstm_output = Bidirectional(lstm_layer)(bilstm_input)

# CNN model
cnn_output = Conv1D(filters=64, kernel_size=3, activation='relu')(cnn_input)
cnn_output = MaxPooling1D(pool_size=2)(cnn_output)
cnn_output = Flatten()(cnn_output)

# Concatenate the outputs of BiLSTM and CNN models
concatenated = Concatenate()([bilstm_output, cnn_output])

# Combine with dense layers
combined = Dense(64, activation='relu')(concatenated)
output = Dense(1)(combined)

# Define the ensemble model
ensemble_model_blstm = Model(inputs=[bilstm_input, cnn_input], outputs=output)

# Compile the model
ensemble_model_blstm.compile(optimizer='adam', loss='mse')

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint("ensemble_model__blstm_checkpoint.h5", monitor='val_loss', save_best_only=True)

# Train the model
history = ensemble_model_blstm.fit([X_train_hyb, X_train_hyb], y_train_hyb, epochs=50, batch_size=256, validation_data=([X_test_bcnn, X_test_bcnn], y_test_hyb), callbacks=[early_stopping, checkpoint])


In [ ]:
# Evaluate the model on the test set
test_loss_bilstm = ensemble_model_blstm.evaluate([X_test_bcnn, X_test_bcnn], y_test_bcnn)

print("Test Loss:", test_loss_bilstm)


### 3. Model Performance

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

models = ['BiLSTM', 'LSTM+CNN', 'BiLSTM+CNN']

# Train MSE values
train_mse = [0.01354, 0.01352, 0.01349]

# Test MSE values
test_mse = [0.013483, 0.013461, 0.013482]

# Plotting
plt.figure(figsize=(10, 6))

bar_width = 0.35
index = np.arange(len(models))

bars1 = plt.bar(index, train_mse, bar_width, color='blue', label='Train MSE')
bars2 = plt.bar(index + bar_width, test_mse, bar_width, color='orange', label='Test MSE')

plt.yscale('log')

plt.xlabel('Models')
plt.ylabel('Mean Squared Error (MSE)')
plt.title('Comparison of Train and Test MSE for Different Models')
plt.legend()
plt.xticks(index + bar_width / 2, models, rotation=45)

for bars in [bars1, bars2]:
    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2, yval, round(yval, 6), va='bottom', ha='center')

plt.tight_layout()
plt.show()


# BiLSTM+CNN results

In [ ]:
predicted_values = ensemble_model_blstm.predict([X_test_bcnn, X_test_bcnn])
predicted_values_inverse = scaler.inverse_transform(predicted_values)

y_test_bcnn_inverse = scaler.inverse_transform(y_test_bcnn)

results_df = pd.DataFrame({'Predicted': predicted_values_inverse.flatten(), 'Actual': y_test_bcnn_inverse.flatten()})

print(results_df)

# LSTM+CNN results

In [ ]:
predicted_values = ensemble_model.predict([X_test_hyb, X_test_hyb])
predicted_values_inverse = scaler.inverse_transform(predicted_values)

y_test_inverse = scaler.inverse_transform(y_test_hyb)


results_df = pd.DataFrame({'Predicted': predicted_values_inverse.flatten(), 'Actual': y_test_inverse.flatten()})

print(results_df)